In [ ]:
import random
import numpy as np
from scipy.stats import halfnorm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon


import time


start_time = time.time()

# making the tree to store the v's
class TreeNode:
    def __init__(self, motion, parent=None):
        self.motion = motion
        self.parent = parent
        self.children = []

    def add_child(self, child_motion):
        child = TreeNode(child_motion, parent=self)
        self.children.append(child)
        return child

    def get_leaf_states(self):
        # Recursively find all leaf nodes
        if not self.children:
            return [self.motion]
        leaves = []
        for child in self.children:
            leaves.extend(child.get_leaf_states())
        return leaves

    def get_all_states(self):
        # Returns all the states in the tree (including repeated positions)
        states = [self.motion]
        for child in self.children:
            states.extend(child.get_all_states())
        return states


class Grid:
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = [[[] for _ in range(cols)] for _ in range(rows)]
        self.obstacles = set()
        self.cell_importance = {}

    def add_obstacle(self, x, y):
        if 0 <= x < self.rows and 0 <= y < self.cols:
            self.obstacles.add((x, y))

    def is_obstacle(self, position):
        return position in self.obstacles

    def get_cell(self, position):
        x, y = position
        return x % self.rows, y % self.cols

    def add_motion(self, motion):
        x, y = self.get_cell(motion[0])
        self.grid[x][y].append(motion)

    def select_cell(self, bias):
        all_cells = [(x, y) for x in range(self.rows) for y in range(self.cols)]
        candidates = [cell for cell in all_cells if self.grid[cell[0]][cell[1]]]

        if not candidates:
            return None

        if random.random() < bias:
            return max(candidates, key=lambda cell: len(self.grid[cell[0]][cell[1]]))
        else:
            return random.choice(candidates)

    def select_motion(self, cell):
        x, y = cell
        motions = self.grid[x][y]
        if not motions:
            return None
        idx = min(int(halfnorm.rvs(scale=len(motions) / 3)), len(motions) - 1)
        return motions[idx]

    def print_grid(self):
        for i, row in enumerate(self.grid):
            print(f"Row {i}: {row}")

    def update_cell_importance(self, cell, importance_value):
        self.cell_importance[cell] = importance_value


def is_valid_motion(position, grid):
    x, y = position
    return 0 <= x < grid.rows and 0 <= y < grid.cols and not grid.is_obstacle(position)


def is_goal_reached(position):
    return position == (3, 3)


def expand_motion(motion, grid, visited):


    possible_moves = [
        (1, 0),   # Right
        (-1, 0),  # Left
        (0, 1),   # Down
        (0, -1),  # Up
        (1, 1),   # Down-right diagonal
        (1, -1),  # Down-left diagonal
        (-1, 1),  # Up-right diagonal
        (-1, -1), # Up-left diagonal
    ]

    x, y = motion[0]

    for dx, dy in possible_moves:
        new_position = (x + dx, y + dy)


        if is_valid_motion(new_position, grid) and new_position not in visited:
            if grid.is_obstacle(new_position):
                continue

            visited.add(new_position)
            control = random.random()
            time = motion[2] + random.uniform(0, 1)
            return (new_position, control, time)

    return None



def KPIECE(iterations, initial_motion, grid, bias=0.75):
    root = TreeNode(initial_motion)
    visited = set()
    visited.add(initial_motion[0])
    grid.add_motion(initial_motion)


    G = grid


    for _ in range(iterations):

        cell = G.select_cell(bias)
        if not cell:
            continue


        motion = G.select_motion(cell)
        if not motion:
            continue

        new_motion = expand_motion(motion, G, visited)
        if new_motion:

            node = root.add_child(new_motion)
            G.add_motion(new_motion)

            if is_goal_reached(new_motion[0]):
                print(f"Goal reached at: {new_motion[0]}")
                return root

        # values given in the paper
        P = 0.5 + 0.3 * random.random()


        G.update_cell_importance(cell, min(P, 1))

    print("No solution found within the iteration limit.")
    return root


# acucally making the grid
rows, cols = 4,4
grid = Grid(rows, cols)

# Add obstacles
obstacles = [(1, 2), (1, 0), (3, 2), (3, 2)]
for obs in obstacles:
    grid.add_obstacle(*obs)

# Print config space
print("Configuration Space:")
for i in range(rows):
    row = []
    for j in range(cols):
        row.append(1 if (i, j) in grid.obstacles else 0)
    print(row)

initial_motion = ((0, 0), 0, 0)  # Start state: (position, control, time)
tree_root = KPIECE(1000, initial_motion, grid)


print("visted paths:")
all_states = tree_root.get_all_states()
for state in all_states:
    print(state)


end_time = time.time()

execution_time_python = end_time - start_time
print(f"Python execution time: {execution_time_python:.4f} seconds")


Configuration Space:
[0, 0, 0, 0]
[1, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
Goal reached at: (3, 3)
visted paths:
((0, 0), 0, 0)
((0, 1), 0.7643208699035511, 0.5744677937725806)
((1, 1), 0.07518536931880604, 0.8755514472518163)
((0, 2), 0.14704295164992298, 0.7176286413480677)
((0, 3), 0.4065966671367949, 1.1292646409712181)
((1, 3), 0.7428339777896751, 1.4270648744679857)
((2, 1), 0.5178444725775463, 1.8585154889553044)
((2, 3), 0.7005663769963852, 2.2831677198365057)
((2, 2), 0.37493424265400954, 1.264096258470981)
((2, 0), 0.05843254720030411, 0.9771174664672237)
((3, 3), 0.9181719036438551, 1.636566045478717)
Python execution time: 0.0093 seconds
